In [2]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd
pd.set_option('display.max_columns', None) # 데이터프레임 컬럼 display

import warnings
warnings.filterwarnings('ignore') # 경고 무시

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

# 함수 설정

In [13]:
def get_result(driver) :
    #--------------------- table 태그 읽기 - 팀 별 1 ~ 5 세트 결과 ---------------------#
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    table = soup.find_all('table')
    set_result = pd.read_html(str(table))[1] # 1 ~ 5 세트 별 득점 결과

    #--------------------- 날짜 정보 읽기 ---------------------#
    date_selector = '#wrp_content > article.wrp_recentgame.wrp_result > table > thead > tr > th'
    date = soup.select_one(date_selector).get_text()
    date = date.replace('\xa0', '').split('/')[0].strip().split()
    date = pd.to_datetime(date[0][:-1] + '/' + date[1][:-1] + '/' + date[2][:-1])

    #
    # 첫 번째 팀 정보 읽기
    #

    #--------------------- 버튼 클릭 - 선수 기록 ---------------------#
    x_path = '//*[@id="wrp_content"]/article[2]/ul/li[2]/a/span' # 선수 기록 버튼
    cursor = driver.find_element(By.XPATH,x_path)
    cursor.click()
    time.sleep(1)

    #--------------------- table 태그 읽기 - 선수 별 출전 세트, 득점, 공격 종합 ---------------------#
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    table = soup.find_all('table')

    team1 = pd.read_html(str(table))[3] # 명단 table
    team1_info = pd.read_html(str(table))[4] # 출전 세트, 득점, 공격 종합 table
    team1_info.columns = ['_'.join(col).strip() for col in team1_info.columns]
    team1 = pd.concat([team1, team1_info], axis = 1)

    #--------------------- 나머지 기록 읽기 ---------------------#
    for _ in range(4) :
        #--------------------- 버튼 클릭 - 다음 기록 ---------------------#
        x_path = '//*[@id="team1"]/div/div[1]/a[2]' # 다음 기록 버튼
        cursor = driver.find_element(By.XPATH,x_path)
        cursor.click()
        time.sleep(1)


        #--------------------- table 태그 읽기 - 다음 기록들 ---------------------#
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        table = soup.find_all('table')

        team1_info = pd.read_html(str(table))[4] # 다음 기록
        team1_info.columns = ['_'.join(col).strip() for col in team1_info.columns]
        team1 = pd.concat([team1, team1_info], axis = 1)


    #
    # 두 번째 팀 정보 읽기
    #

    #--------------------- 버튼 클릭 - 두 번째 팀 ---------------------#
    try :
        x_path = '//*[@id="tab2"]/div[6]/ul/li[2]/a/span' # 두 번째 팀 버튼
        cursor = driver.find_element(By.XPATH,x_path)
    except :
        try :
            x_path = '//*[@id="tab2"]/div[5]/ul/li[2]/a/span' # 두 번째 팀 버튼
            cursor = driver.find_element(By.XPATH,x_path)
        except :
            try :
                x_path = '//*[@id="tab2"]/div[4]/ul/li[2]/a/span' # 두 번째 팀 버튼
                cursor = driver.find_element(By.XPATH,x_path)
            except :
                try :
                    x_path = '//*[@id="tab2"]/div[3]/ul/li[2]/a/span' # 두 번째 팀 버튼
                    cursor = driver.find_element(By.XPATH,x_path)
                except :
                    try :
                        x_path = '//*[@id="tab2"]/div[2]/ul/li[2]/a/span' # 두 번째 팀 버튼
                        cursor = driver.find_element(By.XPATH,x_path)
                    except :
                        x_path = '//*[@id="tab2"]/div[1]/ul/li[2]/a/span' # 두 번째 팀 버튼
                        cursor = driver.find_element(By.XPATH,x_path)
                            
    cursor.click()
    time.sleep(1)

    #--------------------- table 태그 읽기 - 선수 별 출전 세트, 득점, 공격 종합 ---------------------#
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    table = soup.find_all('table')

    team2 = pd.read_html(str(table))[5] # 명단 table
    team2_info = pd.read_html(str(table))[6] # 출전 세트, 득점, 공격 종합 table
    team2_info.columns = ['_'.join(col).strip() for col in team2_info.columns]
    team2 = pd.concat([team2, team2_info], axis = 1)

    #--------------------- 나머지 기록 읽기 ---------------------#
    for _ in range(4) :
        #--------------------- 버튼 클릭 - 다음 기록 ---------------------#
        x_path = '//*[@id="team2"]/div/div[1]/a[2]' # 다음 기록 버튼
        cursor = driver.find_element(By.XPATH,x_path)
        cursor.click()
        time.sleep(1)


        #--------------------- table 태그 읽기 - 다음 기록들 ---------------------#
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        table = soup.find_all('table')

        team2_info = pd.read_html(str(table))[6] # 다음 기록
        team2_info.columns = ['_'.join(col).strip() for col in team2_info.columns]
        team2 = pd.concat([team2, team2_info], axis = 1)
    
    return set_result, date, team1, team2

In [4]:
def clean_result(set_result, date, team1, team2) :
    # No. 열 삭제
    team1 = team1.drop('No.', axis = 1)
    team2 = team2.drop('No.', axis = 1)

    
    # 3경기 이상 출전한 선수만 유지 (출전 세트 결측치가 3개 이상인 선수 제거)
    subset = ['출전세트_1set', '출전세트_2set', '출전세트_3set', '출전세트_4set', '출전세트_5set']

    # thresh = 3 : 정상 값(Not NaN)이 3개 이상만 유지
    team1 = team1.dropna(subset = subset, thresh = 3)
    team2 = team2.dropna(subset = subset, thresh = 3)

    # 스타팅멤버 표시 (3세트 이상 스타팅멤버라면 1 아니면 0)
    team1['스타팅멤버'] = 0
    team2['스타팅멤버'] = 0

    team1.loc[(team1.iloc[:, 1:6] == 'O').sum(axis = 1) >= 3, '스타팅멤버'] = 1
    team2.loc[(team2.iloc[:, 1:6] == 'O').sum(axis = 1) >= 3, '스타팅멤버'] = 1
    
    
    # 출전 세트 열 삭제
    subset = ['출전세트_1set', '출전세트_2set', '출전세트_3set', '출전세트_4set', '출전세트_5set']

    team1 = team1.drop(subset, axis = 1)
    team2 = team2.drop(subset, axis = 1)

    
    # 이름과 포지션 분리하기
    # "이름" 열 : 이름 정보
    # "포지션" 열 : 포지션 정보
    team1['포지션'] = team1['이름'].str.split().str[1].str.strip('()')
    team1['이름'] = team1['이름'].str.split().str[0]

    team2['포지션'] = team2['이름'].str.split().str[1].str.strip('()')
    team2['이름'] = team2['이름'].str.split().str[0]

    
    # 선수 별 득점 점유율 구하기 (개인 별 득점을 팀 전체의 득점으로 나누기)
    team1_total_score = team1['득점_득점'].sum()
    team2_total_score = team2['득점_득점'].sum()
    team1['득점점유율'] = team1['득점_득점'] / team1_total_score
    team2['득점점유율'] = team2['득점_득점'] / team2_total_score
    
    
    # 컬럼 선택
    columns = [
        '이름',
        '득점_득점',
        '공격종합_시도',
        '공격종합_성공',
        '공격종합_공격차단',
        '공격종합_범실',
        '공격종합_성공률',
        '공격종합_점유율',
        '오픈_시도',
        '오픈_성공',
        '오픈_공격차단',
        '오픈_범실',
        '오픈_성공률',
        '오픈_점유율',
        '시간차_시도',
        '시간차_성공',
        '시간차_공격차단',
        '시간차_범실',
        '시간차_성공률',
        '시간차_점유율',
        '이동_시도',
        '이동_성공',
        '이동_공격차단',
        '이동_범실',
        '이동_성공률',
        '이동_점유율',
        '후위_시도',
        '후위_성공',
        '후위_공격차단',
        '후위_범실',
        '후위_성공률',
        '후위_점유율',
        '속공_시도',
        '속공_성공',
        '속공_공격차단',
        '속공_범실',
        '속공_성공률',
        '속공_점유율',
        '퀵오픈_시도',
        '퀵오픈_성공',
        '퀵오픈_공격차단',
        '퀵오픈_범실',
        '퀵오픈_성공률',
        '퀵오픈_점유율',
        '서브_시도',
        '서브_성공',
        '서브_범실',
        '서브_성공률',
        '서브_점유율',
        '디그_시도',
        '디그_성공',
        '디그_실패',
        '디그_범실',
        '디그_세트당',
        '디그_점유율',
        '세트_시도',
        '세트_성공',
        '세트_범실',
        '세트_세트당',
        '세트_점유율',
        '리시브_시도',
        '리시브_정확',
        '리시브_실패',
        '리시브_세트당',
        '리시브_점유율',
        '블로킹_시도',
        '블로킹_성공',
        '블로킹_유효블락',
        '블로킹_실패',
        '블로킹_범실',
        '블로킹_세트당',
        '블로킹_점유율',
        '블로킹_어시스트',
        '벌칙_벌칙',
        '범실_범실',
        '포지션',
        '득점점유율',
        '스타팅멤버'
    ]
    team1 = team1[columns]
    team2 = team2[columns]
    
    
    # 경기날짜, 팀명, 결과 정보 추가
    team1['팀명'] = set_result.iloc[0,0]
    team2['팀명'] = set_result.iloc[1,0]
    team1['결과'] = 1 if int(set_result.iloc[0,-1]) > int(set_result.iloc[1,-1]) else 0
    team2['결과'] = 0 if int(set_result.iloc[0,-1]) > int(set_result.iloc[1,-1]) else 1
    team1['경기날짜'] = date
    team2['경기날짜'] = date

    # 컬럼 순서 변경
    team1 = team1[['팀명', '경기날짜', '결과'] + columns]
    team2 = team2[['팀명', '경기날짜', '결과'] + columns]
    
    
    # 연결
    team = pd.concat([team1, team2])
    team
    
    return team

# 기간 설정

In [5]:
# 반복문으로 각 url 출력
seasons = [f'0{num}' for num in range(14, 19)]
yymms = [
pd.date_range('2017-10', '2018-04', freq = 'M').strftime('%Y-%m').to_list(),
pd.date_range('2018-10', '2019-04', freq = 'M').strftime('%Y-%m').to_list(),
pd.date_range('2019-10', '2020-04', freq = 'M').strftime('%Y-%m').to_list(),
pd.date_range('2020-10', '2021-05', freq = 'M').strftime('%Y-%m').to_list(),
pd.date_range('2021-10', '2022-04', freq = 'M').strftime('%Y-%m').to_list()
]
# print(seasons)
# print(yymms)

for season, dates in zip(seasons, yymms) :
    for date in dates :
        url = f'https://www.kovo.co.kr/game/v-league/11110_schedule_list.asp?season={season}&team=&yymm={date}&r_round='
        print(url)

https://www.kovo.co.kr/game/v-league/11110_schedule_list.asp?season=014&team=&yymm=2017-10&r_round=
https://www.kovo.co.kr/game/v-league/11110_schedule_list.asp?season=014&team=&yymm=2017-11&r_round=
https://www.kovo.co.kr/game/v-league/11110_schedule_list.asp?season=014&team=&yymm=2017-12&r_round=
https://www.kovo.co.kr/game/v-league/11110_schedule_list.asp?season=014&team=&yymm=2018-01&r_round=
https://www.kovo.co.kr/game/v-league/11110_schedule_list.asp?season=014&team=&yymm=2018-02&r_round=
https://www.kovo.co.kr/game/v-league/11110_schedule_list.asp?season=014&team=&yymm=2018-03&r_round=
https://www.kovo.co.kr/game/v-league/11110_schedule_list.asp?season=015&team=&yymm=2018-10&r_round=
https://www.kovo.co.kr/game/v-league/11110_schedule_list.asp?season=015&team=&yymm=2018-11&r_round=
https://www.kovo.co.kr/game/v-league/11110_schedule_list.asp?season=015&team=&yymm=2018-12&r_round=
https://www.kovo.co.kr/game/v-league/11110_schedule_list.asp?season=015&team=&yymm=2019-01&r_round=


# 기간 확인

In [11]:
# 인덱스 맞춰 주기
# 시즌 014~018
for i in range(5):
    season = seasons[i]
    dates = yymms[i]
    print(i, season, dates)

0 014 ['2017-10', '2017-11', '2017-12', '2018-01', '2018-02', '2018-03']
1 015 ['2018-10', '2018-11', '2018-12', '2019-01', '2019-02', '2019-03']
2 016 ['2019-10', '2019-11', '2019-12', '2020-01', '2020-02', '2020-03']
3 017 ['2020-10', '2020-11', '2020-12', '2021-01', '2021-02', '2021-03', '2021-04']
4 018 ['2021-10', '2021-11', '2021-12', '2022-01', '2022-02', '2022-03']


# 기간 선택해서 돌리기

In [14]:
dfs = []
season = seasons[0]
dates = yymms[0]

driver = webdriver.Chrome()

for date in dates :
    # 경기 스케줄 url 접속
    url = f'https://www.kovo.co.kr/game/v-league/11110_schedule_list.asp?season={season}&team=&yymm={date}&r_round='
    driver.get(url)
    time.sleep(1)


    # 경기 스케줄을 데이터프레임으로 저장 후 여자 경기의 인덱스를 idx에 저장
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    table = soup.find_all('table')
    try :
        df = pd.read_html(str(table))[0]
        idx = df[df['구분'] == '여자'].index.to_list()
    except :
        df = pd.read_html(str(table))[2]
        idx = df[df['구분'] == '여자'].index.to_list()
    idx = [i + 1 for i in idx]



    # 각 경기의 상세결과 버튼 클릭
    for i in idx :
        # 상세결과 버튼 클릭
        try :
            x_path = f'//*[@id="type1"]/div/table/tbody/tr[{i}]/td[10]/a[3]' # 상세결과 버튼
            cursor = driver.find_element(By.XPATH,x_path)
        except :
            x_path = f'//*[@id="type1"]/div/table/tbody/tr[{i}]/td[10]/a[2]' # 상세결과 버튼
            cursor = driver.find_element(By.XPATH,x_path)
        cursor = driver.find_element(By.XPATH,x_path)
        cursor.click()
        time.sleep(1)

        # 함수 적용
        set_result, date, team1, team2 = get_result(driver)
        team = clean_result(set_result, date, team1, team2)
        dfs.append(team)

        # 다시 경기 스케줄 url로 이동
        driver.get(url)
        time.sleep(1)
        
# csv 파일로 저장
df = pd.concat(dfs)
df.to_csv(f'volleyball_{season}.csv', index = False)
df = pd.read_csv(f'volleyball_{season}.csv')
df

NameError: name 'By' is not defined